# Write examples

In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

In [ ]:
from pmd_beamphysics import ParticleGroup, pmd_init
from h5py import File
import os

In [ ]:
# Pick one:

# H5File = 'data/bmad_particles2.h5'
H5FILE = "data/distgen_particles.h5"
# H5FILE = 'data/astra_particles.h5'

P = ParticleGroup(H5FILE)

# openPMD

The regular write routine writes in a proper openPMD format

In [ ]:
P.write("openpmd_particles.h5")

An open h5 hande can also be used, but it needs to be properly initialized

In [ ]:
with File("openpmd_particles.h5", "w") as h5:
    pmd_init(h5, basePath="/", particlesPath="/")
    P.write(h5)

This can be read in by another ParticleGroup

In [ ]:
P2 = ParticleGroup("openpmd_particles.h5")

Check that they are the same:

In [ ]:
P2 == P

# Astra

In [ ]:
P.write_astra("astra_particles.txt")

In [ ]:
!head astra_particles.txt

Check the readback:

In [ ]:
from pmd_beamphysics.interfaces.astra import parse_astra_phase_file
import numpy as np

P1 = ParticleGroup(data=parse_astra_phase_file("astra_particles.txt"))
for k in ["x", "px", "y", "py", "z", "pz"]:
    assert np.allclose(P[k], P1[k])

# Bmad ASCII

In [ ]:
P.write_bmad("bmad_particles.txt")

In [ ]:
!head bmad_particles.txt

# Bmad dict

In [ ]:
P.to_bmad()

Check that the conversion preserves information. Note that `==` uses `np.allclose`, because there is roundoff error in the conversion.

In [ ]:
assert P == P.from_bmad(P.to_bmad())

# elegant

In [ ]:
P.write_elegant("elegant_particles.txt", verbose=True)

In [ ]:
!head -n 20 elegant_particles.txt

# Genesis 1.3 v2 

In [ ]:
P.write_genesis2_beam_file("genesis2.beam", n_slice=50, verbose=True)

In [ ]:
!head genesis2.beam

# Genesis 1.3 v4

## beam file (slice statistics)

In [ ]:
input_str = P.write_genesis4_beam(
    "genesis4_beam.h5", n_slice=123, verbose=True, return_input_str=True
)

This string is optionally returned for use in the main Genesis4 input file:

In [ ]:
print(input_str)

These are the datasets written:

In [ ]:
with File("genesis4_beam.h5", "r") as h5:
    for g in h5:
        print(g, len(h5[g]), h5[g].attrs["unitSymbol"])

## Distribution file (particles)

In [ ]:
P.write_genesis4_distribution("genesis4_distribution.h5", verbose=True)

This is what is written:

In [ ]:
with File("genesis4_distribution.h5", "r") as h5:
    for g in h5:
        print(g, len(h5[g]))

# GPT ASCII

In [ ]:
P.write_gpt("gpt_particles.txt", verbose=True)

In [ ]:
if os.path.exists(os.path.expandvars("$ASCI2GDF_BIN")):
    P.write_gpt("gpt_particles.gdf", verbose=True, asci2gdf_bin="$ASCI2GDF_BIN")

In [ ]:
#!head gpt_particles.txt

# Impact-T

Impact-T particles must all be a the same time:

In [ ]:
P.drift_to_t(P["mean_t"])

This will return settings for Impact-T to use:

In [ ]:
P.write_impact("impact_particles.txt")

In [ ]:
!head impact_particles.txt

# LiTrack

LiTrack particles must be at the same z:

In [ ]:
P.drift_to_z()

In [ ]:
P.write_litrack("litrack.zd", verbose=True)

In [ ]:
!head -n 20 litrack.zd

# Lucretia

In [ ]:
P.write_lucretia(
    "lucretia.mat", ele_name="BEGINNING", t_ref=0, stop_ix=None, verbose=True
)

Read back:

In [ ]:
from pmd_beamphysics.interfaces.lucretia import lucretia_to_data, list_element_names

ParticleGroup(data=lucretia_to_data("lucretia.mat", verbose=True))

Helper function to list the available elements:

In [ ]:
list_element_names("lucretia.mat")

# OPAL

Injected particled must be at the same time:

In [ ]:
P.drift_to_t()

P.write_opal("opal_injected.txt", dist_type="injected")

In [ ]:
!head opal_injected.txt

Emitted particles must be at the same z:

In [ ]:
P.drift_to_z(P["mean_z"])
P.write_opal("opal_emitted.txt", dist_type="emitted")

In [ ]:
!head opal_emitted.txt

# SIMION 
Write SIMION input files (*.ion) 

In [ ]:
P.write_simion("simion_particles.ion")

# Cleanup

In [ ]:
for file in [
    "astra_particles.txt",
    "bmad_particles.txt",
    "elegant_particles.txt",
    "gpt_particles.txt",
    "impact_particles.txt",
    "opal_injected.txt",
    "opal_emitted.txt",
    "openpmd_particles.h5",
    "genesis4_beam.h5",
    "genesis4_distribution.h5",
    "genesis2.beam",
    "litrack.zd",
    "gpt_particles.gdf",
    "lucretia.mat",
    "simion_particles.ion",
]:
    if os.path.exists(file):
        os.remove(file)